In [1]:
import os, sys
import torch
import signal
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "true"
from main import Main__

model_config = {'amp':True, 'use_distributed':False,'accum_grad_iters':1,
                'batch_size':1,
                'chat_template': True, 'end_sym': '\n', 'prompt_template': "[INST] {} [/INST]",
                'max_txt_len': 1024, 'max_context_len': 3500,
                'ouput_dir': './llama3_vit_L-clip336', #./llama3_vit_L-clip336, ./llama3_vit_B-clip224-b16
                'stage_ckpt': '/ssd3/chih/LLM/MiniGPT-4-ckpt/checkpoint_stage3.pth', 
                'vis_root_train': './dataset/minigpt_casing_train/coco/image/train',
                'ann_paths_train': ['./dataset/minigpt_casing_train/coco_caption/defe_ready_anno.json'],
                'vis_root_valid': './dataset/minigpt_casing_test/coco/image/test',
                'ann_paths_valid': ['./dataset/minigpt_casing_test/coco_caption/defe_ready_anno.json']}

# llm_config = {'llama_model':'/ssd3/chih/LLM/Llama-2-7b-chat-hf', 'low_resource':False, 'low_res_device':0, 
#               'lora_r':64, 'lora_target_modules':["q_proj", "v_proj"], 'lora_alpha':16,'lora_dropout':0.05
#               }
llm_config = {'llama_model':'/ssd3/chih/LLM/Meta-Llama-3-8B-Instruct', 'low_resource':True, 'low_res_device':0, 
              'lora_r':64, 'lora_target_modules':["q_proj", "v_proj"], 'lora_alpha':16,'lora_dropout':0.05
              }
# '/ssd3/chih/LLM/Meta-Llama-3-8B-Instruct'


vit_config = {'model_name':'clip_large_336', #eva_clip_g, clip_large_336
              'model_path':"../../VITModel/clip-vit-large-patch14-336", #clip-vit-base-patch16, clip-vit-large-patch14-336
              'image_size': 336,  #bilp2 = 448, clip = 224 or 336
              'drop_path_rate': 0, 'use_grad_checkpoint': True, 'vit_precision': 'fp16', 'freeze_vit': True, }

lr_config = {'init_lr': 1e-5, 'beta2':0.999,'min_lr': 1e-6, 'decay_rate': None, 'weight_decay':0.05,
                'warmup_start_lr': 1e-6, 'warmup_steps': 1000, 'iters_per_epoch': 1000}


main_ = Main__(model_config=model_config)
main_.VLM_build(llm_config=llm_config, vit_config=vit_config)

Loading LLAMA


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer pass


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 27,262,976 || all params: 8,057,524,224 || trainable%: 0.3384


/ssd3/chih/miniconda3/envs/minigptv/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
from utils.caption_datasets import COCOCaptionDataset
from torch.utils.data import DataLoader
# demo測試對話反饋
train_data_set = COCOCaptionDataset(vis_root = model_config['vis_root_train'], 
                                ann_paths = model_config['ann_paths_train'],
                                img_size = vit_config['image_size'])
dataloader = DataLoader(train_data_set, batch_size=1, num_workers=5, shuffle=True, pin_memory=True)
ckpt_url = '/ssd3/chih/branch/VLM_Rebuild/llama3_vit_L-clip336/checkpoint_best.pth' 
main_.load_checkpoint(ckpt_url=ckpt_url)

ann paths ['./dataset/minigpt_casing_train/coco_caption/defe_ready_anno.json']
{'caption': 'Defective.', 'image': '/ssd3/chih/Dataset/minigpt_casing_train/coco/image/train/casting512x512_deffront_0.jpeg', 'image_id': 'casting512x512_deffront_0.jpeg', 'instance_id': '0'}
./dataset/minigpt_casing_train/coco/image/train/casting512x512_deffront_129.jpeg 



In [3]:
main_.model.llama_model.generation_config.pad_token_id = main_.model.llama_tokenizer.pad_token_id

In [4]:
samples = next(iter(dataloader))
samples['image'] = samples['image'].cuda()
ans = main_.chat_module(samples=samples)
print("讀取使用fine tune後的llama3權重直接輸出 [同資料集finetune]")
print("Q:",samples['instruction_input'], "answer:",samples['answer'])
print("A:",ans)

<|begin_of_text|>100
flawless
no
flawless
flawless
flawless

讀取使用fine tune後的llama3權重直接輸出 [同資料集finetune]
Q: ['<Img><ImageHere></Img> [caption] Please answer Defective product or Flawless product. '] answer: ['no']
A: ['<|begin_of_text|>100\nflawless\nno\nflawless\nflawless\nflawless']
